In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Будем парсить случайные статьи в Википедии (https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%A1%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0)
Из статьи будем доставать следующую информацию:

-url статьи (str)

-название (str)

-краткая информация (str)

-языки (list of str)

-разделы (list of str)	

-разделы в инфобоксе (list of str)	

-количество примечаний (int)	

-последнее редактирование (str)	

-категории (list of str)	

-количество внешних ссылок (int)

In [24]:
def get_info():
    url = "https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%A1%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0"
    g = requests.get(url)
    page = BeautifulSoup(g.text, "html.parser")
    
    title = page('title')[0].string[:-12] #название статьи

    links = len([link['href'] for link in page.findAll("a", class_='external')]) #количество внешних ссылок

    description = "" #краткое описание
    for i in page.findAll('div', class_='mw-parser-output')[0]:
        if i.name == 'p':
            description = i.get_text()
            break

    address = g.url #адрес статьи

    date = page.findAll('li', id="footer-info-lastmod")[0].string[51:-1] #последнее редактирование

    temp_prim = page.findAll('ol', class_='references') #количество "примечаний"
    if(len(temp_prim) != 0):
        prim = len(temp_prim[0]('li'))
    else:
        prim = 0

    cats = [i.string for i in page.findAll('div', id='catlinks')[0]('ul')[0]('a')] #категории

    temp_langs = page.findAll('nav', id='p-lang') #языки
    if(len(temp_langs) != 0):
        langs = [i.string for i in page.findAll('nav', id='p-lang')[0]('li')]
    else:
        langs = []

    toms = [i.string for i in page.findAll('span', class_='mw-headline')] #разделы

    infos = [i.string for i in page.findAll('th', class_='infobox-header')] #количество разделов в "инфобоксе"
    
    return (address, title, description, langs, toms, infos, prim, date, cats, links)

In [26]:
get_info()

('https://ru.wikipedia.org/wiki/%D0%91%D0%B0%D0%B1%D0%B8%D0%BA%D0%BE%D0%B2%D0%BE_(%D0%91%D0%B0%D1%88%D0%BA%D0%BE%D1%80%D1%82%D0%BE%D1%81%D1%82%D0%B0%D0%BD)',
 'Бабиково (Башкортостан)',
 'Бабиково (башк. Бабик) — деревня в Чишминском районе Башкортостана, относится к Кара-Якуповскому сельсовету. \n',
 ['تۆرکجه',
  'Башҡортса',
  'Нохчийн',
  'Чӑвашла',
  'English',
  'Հայերեն',
  'Қазақша',
  'Татарча/tatarça',
  'Українська',
  'Bân-lâm-gú'],
 ['Население', 'Географическое положение', 'Примечания', 'Ссылки'],
 ['История и география', 'Население', 'Цифровые идентификаторы'],
 3,
 '8 апреля 2020 в 14:44',
 ['Населённые пункты по алфавиту', 'Населённые пункты Чишминского района'],
 17)

In [27]:
lst = []
for i in range(100):
    lst.append(get_info())
df = pd.DataFrame(lst)
df

,0,1,2,3,4,5,6,7,8,9
0,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D1%...,"Берлов, Михаил Николаевич",Михаил Иванович Берлов (1867—1935) — русский и...,[],"[Биография, Семья, Примечания, Литература, Ссы...",[],2,23 апреля 2020 в 07:49,"[Родившиеся 3 июня, Родившиеся в 1867 году, Пе...",5
1,https://ru.wikipedia.org/wiki/%D0%A2%D0%B8%D0%...,Типчак (БПЛА),"1К133 ""Типчак"" — комплекс для ведения воздушно...",[],"[История, Состав комплекса, Характеристики БПЛ...",[],7,2 мая 2020 в 15:32,"[Воздушные суда по алфавиту, Воздушные суда, о...",14
2,https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%...,Резец (инструмент),"Резе́ц — это режущий инструмент, предназначен...","[العربية, Azərbaycanca, Беларуская, Čeština, D...","[Элементы токарного резца, Углы резца и их наз...",[],1,29 августа 2020 в 23:00,[Режущие инструменты],10
3,https://ru.wikipedia.org/wiki/%D0%A1%D0%BA%D0%...,Скобельцыно,Скобе́льцыно — село в Архаринском районе Амурс...,"[Нохчийн, Тоҷикӣ, Українська]","[География, Инфраструктура, Примечания, Ссылки]","[История и география, Цифровые идентификаторы]",0,26 апреля 2020 в 04:48,"[Населённые пункты по алфавиту, Населённые пун...",18
4,https://ru.wikipedia.org/wiki/%D0%95%D0%B2%D1%...,Евровидение-1976,Конкурс песни Евровидение 1976 — 21-й конкурс ...,"[Azərbaycanca, Беларуская, Беларуская (тарашке...","[Место проведения, Формат, Участвующие страны,...","[Даты, Проведение, Участники, Результаты, Евро...",12,12 декабря 2020 в 11:44,"[Евровидение-1976, 1976 год в музыке]",10
...,...,...,...,...,...,...,...,...,...,...
95,https://ru.wikipedia.org/wiki/%D0%9E%D1%81%D0%...,"Осен, Августа","Августа Осен (норв. Augusta Aasen, 19 мая 1878...","[English, Հայերեն, Norsk bokmål]","[Биография, Литература]",[],0,17 февраля 2018 в 10:07,"[Родившиеся 19 мая, Родившиеся в 1878 году, Ро...",0
96,https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%...,Протестантизм в Танзании,Протестантизм в Танзании — одно из направлений...,[],"[Исторический обзор, Современное состояние, Пр...",[],21,27 октября 2019 в 11:05,"[Протестантизм по странам, Христианство в Танз...",31
97,https://ru.wikipedia.org/wiki/Clonostachys_rosea,Clonostachys rosea,Clonóstachys rósea (лат.) — вид аскомицетовых ...,"[Cebuano, Deutsch, English, Suomi, 日本語, Nederl...","[Описание, Экология и значение, Таксономия, Си...","[Научная классификация, Международное научное ...",0,10 января 2020 в 12:33,"[Грибы по алфавиту, Гипокрейные, Виды грибов, ...",4
98,https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%...,Научно-исследовательский институт «Ройан» (Иран),Научно-исследовательский институт «Ройан» (пер...,"[English, Español, فارسی]","[Деятельность института, Научная эффективность...",[],3,4 декабря 2020 в 22:13,[Научные институты Ирана],10


In [34]:
df.columns = ['url', 'название', 'краткое описание', 'языки', 'разделы', 'разделы в табличке', 'количество примечаний', 'последнее редактирование', 'категории', 'количество внешних ссылок']

In [37]:
for i in df['языки']:
    i.append('Русский')#когда парсим языки, не учитывается язык, на котором открыта статья

In [38]:
df.to_excel("output.xlsx") #создаем эксель табличку